In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
%pylab inline
from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge
from numpy.linalg import norm
from numpy import copy

from scipy.stats import probplot
import scipy as sp

from ipywidgets import FloatProgress
from IPython.display import display

import timeit

import fibonacci_heap_mod as fhm

from scipy.sparse import csr_matrix
from scipy import sparse

import sys

Populating the interactive namespace from numpy and matplotlib


## Спарсификация исходных данных

In [56]:
import numpy as np
from scipy import sparse

class SparsificationException(Exception):
    pass


def arora_sparsification(M, e = None, random_seed = None):
    if random_seed is not None:
        np.random.seed(random_seed)

    if e is None:
        e = M.max()

    A = sparse.lil_matrix(M.copy())

    for i, j in zip(*A.nonzero()):
        x = A[i, j]
        if abs(x) > e:
            continue
        p = abs(x)/e
        if p > 1 or p < 0:
            raise SparsificationException("Inadequate probability on (%d, %d): %.2f"%(i, j, p))
        if np.random.rand() <= p:
            A[i, j] = np.sign(x)*e
        else:
            A[i, j] = 0

    return A

## Тест быстродействия

In [68]:
%%time

n = 1e6
m = n
mu = 1/n
X = sparse.rand(m, n, density=2/n)
x_true = np.random.rand(int(n))
x_true /= x_true.sum()
y = X.dot(x_true.T)
e_lower_lim = X.max()
print(len(X.nonzero()[0]))
start_point = np.random.rand(int(n))
start_point /= sum(start_point)

2000000
CPU times: user 7.32 s, sys: 249 ms, total: 7.57 s
Wall time: 7.67 s


In [69]:
%%time

X_b = arora_sparsification(X, e_lower_lim*5)

CPU times: user 39.1 s, sys: 998 ms, total: 40.1 s
Wall time: 41.1 s


In [70]:
print("%d of non-zero elements before"%(len(X.nonzero()[0])))

2000000 of non-zero elements before


In [71]:
print("%d of non-zero elements after"%(len(X_b.nonzero()[0])))

199576 of non-zero elements after
